In [1]:
%matplotlib inline
import re
import pandas as pd
import numpy as np
import nltk
import tqdm as tqdm
import string
from nltk.corpus import stopwords
stop = stopwords.words("english")
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

!pip install gensim
import gensim
from gensim import summarization
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from math import floor,ceil

from sklearn.svm import LinearSVC


import seaborn as sns

In [2]:
data = pd.read_csv('dataset2.csv',encoding='latin-1')

In [3]:
data.head()

,tweets
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,is upset that he can't update his Facebook by ...
2,@Kenichan I dived many times for the ball. Man...
3,my whole body feels itchy and like its on fire
4,"@nationwideclass no, it's not behaving at all...."


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530972 entries, 0 to 530971
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   tweets  530972 non-null  object
dtypes: object(1)
memory usage: 4.1+ MB


In [5]:
data.describe()

,tweets
count,530972
unique,524891
top,isPlayer Has Died! Sorry
freq,210


In [6]:
## Cleaning the Data

In [7]:
def cleaning(tweet , remove_stopwords = True):
    text = re.sub("[^a-zA-Z]"," ", str(tweet))
    words =text.lower().split()
    
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    b=[]
    stemmer = english_stemmer 
    for word in words:
        b.append(stemmer.stem(word))

    
    return(b)

    
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt


In [8]:
## Testing the cleaning function

In [9]:
cleaning("@@//http://www.")

['http', 'www']

In [10]:
## applying the cleaning funciton to the whole data

In [11]:
clean_Text = []
for review in data['tweets']:
    clean_Text.append( " ".join(cleaning(review)))
    

In [12]:
clean_Text

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day',
 'upset updat facebook text might cri result school today also blah',
 'kenichan dive mani time ball manag save rest go bound',
 'whole bodi feel itchi like fire',
 'nationwideclass behav mad see',
 'kwesidei whole crew',
 'need hug',
 'loltrish hey long time see yes rain bit bit lol fine thank',
 'tatiana k nope',
 'twittera que muera',
 'spring break plain citi snow',
 'pierc ear',
 'caregiv bear watch thought ua loss embarrass',
 'octolinz count idk either never talk anymor',
 'smarrison would first gun realli though zac snyder doucheclown',
 'iamjazzyfizzl wish got watch miss iamlilnicki premier',
 'holli death scene hurt sever watch film wri director cut',
 'file tax',
 'lettya ahh ive alway want see rent love soundtrack',
 'fakerpattypattz oh dear drink forgotten tabl drink',
 'alydesign day get much done',
 'one friend call ask meet mid valley today time sigh',
 'angri barista bake cake ate',
 'week 

In [13]:
## forming the tf-idf matrix

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=4, max_features = 10000)
fittedvz = vectorizer.fit(clean_Text)
vz = fittedvz.transform(clean_Text)
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print("home: " + str(tfidf["home"]))
print("like: " + str(tfidf["like"]))
print("tast: " + str(tfidf["tast"]))
print("good: " + str(tfidf["good"]))
print("bad: " + str(tfidf["bad"]))

home: 4.501435186471618
like: 3.965080984354871
tast: 7.629958564439082
good: 4.3121738037951864
bad: 4.589588476033031


In [15]:
vz.shape

(530972, 10000)

In [16]:
## importing the vader sentence sentiment analyser

In [17]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
Senti = SentimentIntensityAnalyzer()
sample_review = clean_Text[:5]
for sentence in sample_review:
    sentence
    ss = Senti.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]))
    print(sentence) 

compound: -0.3818, 
neg: 0.178, 
neu: 0.822, 
pos: 0.0, 
switchfoot http twitpic com zl awww bummer shoulda got david carr third day
compound: -0.4588, 
neg: 0.308, 
neu: 0.692, 
pos: 0.0, 
upset updat facebook text might cri result school today also blah
compound: 0.4939, 
neg: 0.0, 
neu: 0.738, 
pos: 0.262, 
kenichan dive mani time ball manag save rest go bound
compound: 0.0258, 
neg: 0.27, 
neu: 0.449, 
pos: 0.281, 
whole bodi feel itchi like fire
compound: -0.4939, 
neg: 0.516, 
neu: 0.484, 
pos: 0.0, 
nationwideclass behav mad see


In [18]:
## making the k means clustering model

In [19]:
from sklearn.cluster import MiniBatchKMeans

num_clusters = 3
kmeans_model = MiniBatchKMeans(n_clusters=num_clusters, init='k-means++', n_init=1, 
                         init_size=1000, batch_size=1000, verbose=False, max_iter=1000)
kmeans = kmeans_model.fit(vz)

In [20]:
kmeans_clusters = kmeans.predict(vz)
kmeans_distances = kmeans.transform(vz)
sorted_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i)
    for j in sorted_centroids[i, :30]:
        print(' %s' % terms[j])
    print()

Cluster 0:
 work
 miss
 get
 day
 feel
 like
 today
 want
 sad
 good
 back
 realli
 know
 got
 time
 still
 wish
 one
 sorri
 need
 think
 bad
 oh
 hate
 home
 last
 http
 lol
 love
 sleep

Cluster 1:
 go
 want
 work
 back
 home
 bed
 get
 wanna
 today
 day
 sleep
 tomorrow
 school
 realli
 wish
 miss
 see
 time
 like
 need
 feel
 away
 tonight
 think
 well
 rain
 still
 one
 hour
 got

Cluster 2:
 im
 watch
 sad
 get
 go
 miss
 gonna
 lol
 movi
 work
 like
 bore
 sorri
 cant
 tire
 realli
 want
 feel
 day
 sick
 think
 still
 know
 dont
 love
 good
 got
 show
 home
 last



In [21]:
## clearly the cluster 0 is postive , cluster 1 is negative and cluster 2 is neutral

In [22]:
## testing out the k means model

In [23]:
input = 'switchfoot http twitpic com zl awww bummer shoulda got david carr third day'

In [24]:
input_arr = []
input_arr.append(input)

In [25]:
input_arr

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day']

In [26]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(min_df=1, max_features = 10000)
# vz = vectorizer.fit_transform(input_arr)

In [27]:
# import pickle

In [28]:
# pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))
# pickle.dump(kmeans, open("model_2.pkl", "wb"))

In [29]:
# pickle.dump(cleaning , open("cleaning_tweets.pkl" , "wb"))

In [30]:
import pickle

In [31]:
pickle.dump(fittedvz , open("final_vectorizer.pkl" , "wb"))

In [32]:
pickle.dump(kmeans , open("final_kmeans.pkl" , "wb"))

In [33]:
from sklearn.metrics import confusion_matrix


In [4]:
def cleaning(tweet , remove_stopwords = True):
    text = re.sub("[^a-zA-Z]"," ", str(tweet))
    words =text.lower().split()
    
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    b=[]
    stemmer = english_stemmer 
    for word in words:
        b.append(stemmer.stem(word))

    
    return(b)


import pickle
pickle.dump(cleaning , open("cleaning_new.pkl" , "wb"))